# Exercise 1 - Grid World Hellow World
*Written by Dr. Hanan Shteingart*
## Yandex Data Science School RL Course 2019

In this exercise you will get up to speed with gym.ai a testbed for RL agents.

Use the following lines to use tensorboard in colab:

```
!pip install tensorboardX
from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path='./runs')
```


# Clif Climbing!

> "You send a robot to the wall in order to save the world from the white-walkers.  Your robot needs to go from "S" - your starting location to "G" where dragon glass is hidden. But notice the cliff! Don't fall there"

<img style="float: right;" src="https://snag.gy/S63KeB.jpg" alt="Drawing" width="300" />

This is a standard undiscounted, episodic task, with start and goal states, and the usual actions causing movement up, down, right, and left. Reward is −1 on all transitions except those into the region marked “The Cliff.” Stepping into this region incurs a reward of −100 and sends the agent instantly
back to the start (example 6.6 in RL book by Sutton and Burto, edition 2)

![alt text](https://i.snag.gy/v1hfRK.jpg)


## Part 1 - Implement the Environment (World) in the Gym Package

Gym is a python package use to evaluate RL algorithm (read all about it here https://gym.openai.com/docs/#background-why-gym-2016)

In Gym one can create an Environment `env` and iteract with it.

Basic Concept:
* Action Space - space of possible actions - in our case top,bottom,left,right (total 4)
* Observation Space - space of possible observation. In our case, we have a "fully observed MDP" which means observation==state (12x4 grid, total 48) 

At first, the enviornment is reset via `env.reset()` which return the first observation.
Thereafter, there is a cycle where the environement receives an action chosen by the agent and returns a tuple with the resultant new observation, immidiate reward, whether the episode is over and an unused info variable (the tuple `new_state, reward, is_done, _` where the latter info is not used).

The command env.render() displays the current state of the environment.

**TASK** : Implement the cliff-walking environment as a class inheriting from `gym.Env`. You can use the following link to guide you:
* to learn more about spaces: http://gym.openai.com/docs/#spaces
* https://github.com/openai/gym/blob/master/docs/creating-environments.md
* https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html 
* https://stackoverflow.com/questions/45068568/how-to-create-a-new-gym-environment-in-openai


In [0]:
import gym
from gym import spaces
import numpy as np

class CliffWalking(gym.Env):
    CLIFF_REWARD = -100
    STEP_REWARD = -1
    N_ROWS = 4
    N_COLS = 12
    START = (N_ROWS-1, 0)
    GOAL = (N_ROWS - 1, N_COLS-1)
    ACTION = {0: (0, -1), 1:(0, 1) , 2:(-1, 0) , 3: (1, 0)}
    ACTION_NAME = {0: "LEFT", 1: "RIGHT", 2: "DOWN", 3: "UP"}
    CLIFF_ROW = 0

    def __init__(self):
        super().__init__()
        self.state = None
        self.action_space =  list(self.ACTION.keys())

        self.observation_space =  np.full(shape=self.N_ROWS * self.N_COLS, fill_value=self.STEP_REWARD)

        for clif_index in range(1, self.N_COLS-1):
            actual_index = self.state_to_int((self.N_ROWS-1, clif_index))
            self.observation_space[actual_index] = self.CLIFF_REWARD



        self.last_action = None
        self.last_reward = None
        self.is_done = None

    def step(self, action):

        current_action_values = self.ACTION[action]


        next_row = max(0, self.state[0] + current_action_values[0])
        next_row = min(next_row, self.N_ROWS -1)

        next_col = max(0, self.state[1] + current_action_values[1])
        next_col = min(next_col, self.N_COLS - 1)

        self.state = (next_row, next_col)

        self.last_action = self.ACTION_NAME[action]
        self.last_reward = self.observation_space[self.state_to_int(self.state)]

        self.is_done = self.state == self.GOAL or self.last_reward == self.CLIFF_REWARD
        reward = self.last_reward

        return (self.state_to_int(self.state), reward, self.is_done, {})

    def reset(self):
        self.state = self.START
        self.last_action = None
        self.last_reward = None
        self.is_done = None

        return self.state_to_int(self.state)

    def state_to_int(self, state):
        # helper function returns an integer representative of the coordinate
        return np.ravel_multi_index(state, (self.N_ROWS, self.N_COLS))

    def render(self):
        print('***************************')
        print('a ={0} r={1} done={2}'
              .format(self.last_action, self.last_reward, self.is_done))

        for row_index in  range(self.N_ROWS):
            row_vals =[]
            for col_index in range(self.N_COLS):

                index = self.state_to_int((row_index, col_index))
                value = self.observation_space[index]

                if(row_index, col_index) == self.state:
                    row_val = 'P'
                elif (row_index, col_index) == self.START:
                    row_val = 'S'
                elif (row_index, col_index) == self.GOAL:
                    row_val = 'G'
                else:
                    if value ==  self.CLIFF_REWARD:
                        row_val = 'C'
                    else:
                        row_val = 'R'

                row_vals.append(row_val)

            print(','.join(row_vals))

In [2]:
#demo run
env = CliffWalking()
env.reset()
env.render()
env.step(0)
env.render()
env.step(3)
env.render()
env.step(3)
env.render()
env.step(3)
env.render()
env.step(3)
env.render()
env.step(3)
env.render()
env.step(3)
env.render()

env.state
for _ in range(11):
  env.step(1)
  env.render()
env.step(2)
env.render()
env.step(2)
env.render()
env.step(2)
env.render()

***************************
a =None r=None done=None
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
P,C,C,C,C,C,C,C,C,C,C,G
***************************
a =LEFT r=-1 done=False
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
P,C,C,C,C,C,C,C,C,C,C,G
***************************
a =UP r=-1 done=False
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
P,C,C,C,C,C,C,C,C,C,C,G
***************************
a =UP r=-1 done=False
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
P,C,C,C,C,C,C,C,C,C,C,G
***************************
a =UP r=-1 done=False
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
P,C,C,C,C,C,C,C,C,C,C,G
***************************
a =UP r=-1 done=False
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
P,C,C,C,C,C,C,C,C,C,C,G
***************************
a =UP r=-1 done=False
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R,R
R,R,R,R,R,R,R,R,R,R,R

# Q-learning
Recall the Q learning update rule:

![alt text](https://cdn-images-1.medium.com/max/1280/0*BPeyfQgVvGtB7E5U.png)

**TASK**: Implement an on-policy Q-learning agent class `Qlearn` with epsilon greedy decision rule where $\epsilon=0.1$ (with probability $1-\epsilon$ choose optimal action, otherwise random).

**TASK** implement a simulation method `play_episode(agent, env)` to play the agent against the environment and return the total reward.

** TASK** run the play_episode for N_EPISODES = 1000 and log the performance using tensorboard. 

![alt text](https://i.snag.gy/wRJCYG.jpg)

** TASK** Plot the resultant optimal policy learned using `plt.quiver`, e.g.

![alt text](https://i.snag.gy/O915hf.jpg)

**OPTIONAL**: run the learning several times and plot the distribution of reward vs time (`plt.fill_between`)

**OPTIONAL**: study the effect of learning rate, exploration, gamma.
![alt text](https://i.snag.gy/TJE4Gx.jpg)





In [0]:
import gym
import collections
from tensorboardX import SummaryWriter
import numpy as np
GAMMA = 1 # no discounting in this world
ALPHA = 0.1
EPSILON = 0.1

class Qlearning:
    def __init__(self, n_s, n_a, gamma=GAMMA, alpha=ALPHA, epsilon=EPSILON):
        # self.q_values = np.full(shape = (n_s, n_a), fill_value= 0.001)
        self.q_values = np.zeros(shape=(n_s, n_a))
        self.epsilon = epsilon
        self.last_action = None
        self.alpha = alpha
        self.gamma = gamma
        self.last_state = None
        self.last_reward = None

        # TODO
    def act(self, state):

        action = self.random_action() if np.random.random()  < self.epsilon else self.optimal_action(state)
        self.last_action = action
        self.last_state = state

        return action

    def update(self, state, reward):

        self.q_learn_update(self.last_state, self.last_action, reward, state)


        #self.last_reward = reward

    def state_to_int(self, state):
        # helper function returns an integer representative of the coordinate
        return np.ravel_multi_index(state, (self.N_ROWS, self.N_COLS))

    def q_learn_update(self, state, action, reward, new_state):



        old_value = self.q_values[state, action]

        next_max_value = np.max(self.q_values[new_state])

        new_value = old_value * (1 - self.epsilon) + ((self.epsilon) * (reward + self.gamma * next_max_value))

        self.q_values[state, action] = new_value

    def step(self, state, reward):
        self.t += 1
        self.update(state, reward)
        action = self.act(state)

        return action

    def random_action(self):
        action = np.random.choice(self.q_values.shape[1])
        return action

    def optimal_action(self, state):
        state_actions = self.q_values[state]
        action = np.argmax(state_actions)
        return action


      

In [0]:
from tensorboardcolab import TensorBoardColab
LOG_DIR = './runs'
tbc=TensorBoardColab(graph_path=LOG_DIR)

In [0]:
MAX_EPISODES = 1000
RENDER = False
def play_episode(agent, env, render=False):

    current_state = env.reset()
    total_reward = 0.0

    is_done = False
    while not is_done:

        action = agent.act(current_state)

        current_state, reward, is_done, _ = env.step(action)

        agent.update(current_state, reward)

        total_reward += reward
        if render:
            env.render()

    return total_reward

def run(env_args={}, agent_args={}):
  env = CliffWalking(**env_args)
  agent = Qlearning(n_a= len(env.action_space), n_s= env.observation_space.size, **agent_args)
  writer = SummaryWriter(comment="cliff_walking_q_learning", log_dir=LOG_DIR+"/"+ str(env_args)+str(agent_args))
  best_reward = 0
  run_reward = 0

  for i_episode in range(MAX_EPISODES):
    total_reward = play_episode(agent, env, False)
    writer.add_scalar('reward', total_reward, i_episode)

  writer.close()
  return agent, env, run_reward



In [0]:
# plot policy using quiver
%pylab inline
agent, env, reward  = run()
optimal_action = #TODO
optimal_move = #TODO
plt.quiver(np.arange(env.N_COLS), np.arange(env.N_ROWS), 
           optimal_move[0]/2, optimal_move[1]/2)

In [0]:
#study effect of epsilon and alpha
for eps in [0, 0.01, 0.1]:
  for alpha in [0.01, 0.1]:
    _,_,reward = run(agent_args={'epsilon': eps, 'alpha': alpha})
    print('eps={}, alpha={}: reward={}'.format(eps, alpha, reward))